In [1]:
!git clone https://github.com/LizaDerb/probing_compositionality/

Cloning into 'probing_compositionality'...
remote: Enumerating objects: 716, done.
remote: Counting objects: 100% (491/491), done.
remote: Compressing objects: 100% (410/410), done.
remote: Total 716 (delta 149), reused 270 (delta 76), pack-reused 225
Receiving objects: 100% (716/716), 269.38 MiB | 15.23 MiB/s, done.
Resolving deltas: 100% (177/177), done.
Updating files: 100% (152/152), done.


# Подготовка данных

In [2]:
import pandas as pd

In [3]:
corpus = pd.read_csv('/content/probing_compositionality/data/rus/corpus.csv', sep=';')
description = pd.read_csv('/content/probing_compositionality/data/rus/data_description.csv', sep=';')

In [4]:
corpus['mwe'].unique()

array(['себе на голову', 'против течения', 'осиное гнездо',
       'не на своем месте', 'второй дом', 'по барабану', 'жирно будет',
       'гладить по голове', 'за пояс заткнут', 'каши не свариш',
       'с пеной у рта', 'через голову', 'под каблуком', 'удар ниже пояса',
       'поезд ушел', 'смотреть в лицо', 'куда ветер дует',
       'поставить точку', 'на дне', 'смотреть в глаза', 'с блеском',
       'балансировать на грани', 'на мою голову', 'наш брат',
       'голова болит', 'не за горами', 'посмотреть в лицо',
       'косой взгляд', 'на свою голову'], dtype=object)

In [5]:
types = description[['russian', 'type']].set_index('russian').to_dict()['type']

In [6]:
types_col = []
for index, row in corpus.iterrows():
    try:
        types_col.append(types[row['mwe']])
    except:
        if row['mwe'] == 'не на своем месте':
            types_col.append('PP')
        elif row['mwe'] == 'за пояс заткнут':
            types_col.append('VP')
        elif row['mwe'] == 'каши не свариш':
            types_col.append('VP')
        elif row['mwe'] == 'поезд ушел':
            types_col.append('CLAUSE')
        else:
            types_col.append(None)

In [7]:
corpus['type'] = types_col

In [8]:
corpus = corpus[['mwe', 'label', 'sentence', 'type']]
corpus.head()

,mwe,label,sentence,type
0,себе на голову,I,-- Сейчас покажу. Вчера получила -- вчера и п...,OTHER
1,себе на голову,I,"отока радости, текущего вином в дженнете*!"" --...",OTHER
2,себе на голову,I,-- Сейчас покажу. Вчера получила -- вчера и п...,OTHER
3,себе на голову,I,"отока радости, текущего вином в дженнете*!"" --...",OTHER
4,себе на голову,L,"Я почувствовал, что дальше не выдержу. Какие-...",OTHER


In [9]:
corpus['mwe'].value_counts()

mwe
на дне                    1253
через голову               334
смотреть в глаза           231
с блеском                  218
против течения             141
с пеной у рта               86
поставить точку             83
себе на голову              68
по барабану                 57
второй дом                  47
не на своем месте           45
смотреть в лицо             38
осиное гнездо               37
под каблуком                35
поезд ушел                  34
на свою голову              34
куда ветер дует             30
гладить по голове           30
удар ниже пояса             22
жирно будет                 12
наш брат                    10
за пояс заткнут              9
каши не свариш               6
голова болит                 6
посмотреть в лицо            6
на мою голову                5
не за горами                 5
косой взгляд                 4
балансировать на грани       2
Name: count, dtype: int64

In [10]:
corpus = corpus[corpus['mwe'] != 'на дне']

In [11]:
corpus['type'].value_counts()

type
PP        955
VP        417
NP        120
OTHER      73
CLAUSE     70
Name: count, dtype: int64

In [12]:
corpus.groupby('type')['mwe'].value_counts()

type    mwe                   
CLAUSE  поезд ушел                 34
        куда ветер дует            30
        голова болит                6
NP      второй дом                 47
        осиное гнездо              37
        удар ниже пояса            22
        наш брат                   10
        косой взгляд                4
OTHER   себе на голову             68
        не за горами                5
PP      через голову              334
        с блеском                 218
        против течения            141
        с пеной у рта              86
        по барабану                57
        не на своем месте          45
        под каблуком               35
        на свою голову             34
        на мою голову               5
VP      смотреть в глаза          231
        поставить точку            83
        смотреть в лицо            38
        гладить по голове          30
        жирно будет                12
        за пояс заткнут             9
        каши не сва

In [13]:
idiom_for_test = ['голова болит', 'удар ниже пояса', 'наш брат', 'косой взгляд', 'не за горами',
                  'не на своем месте', 'под каблуком', 'на свою голову', 'на мою голову',
                  'жирно будет', 'за пояс заткнут', 'каши не свариш', 'гладить по голове',
                  'балансировать на грани', 'против течения']

In [14]:
corpus_train = corpus[corpus['mwe'].isin(idiom_for_test) == False]
corpus_test = corpus[corpus['mwe'].isin(idiom_for_test)]

In [15]:
print('train: ', len(corpus_train))
print('test: ', len(corpus_test))

train:  1269
test:  366


In [16]:
stat_train_corpus = corpus_train['label'].value_counts().rename('num').to_frame()
stat_train_corpus['percentage'] = (stat_train_corpus.num * 100 / stat_train_corpus.num.sum()).round(1).astype(str) + '%'
stat_train_corpus

,num,percentage
label,,
I,651,51.3%
L,618,48.7%


In [17]:
stat_test_corpus = corpus_test['label'].value_counts().rename('num').to_frame()
stat_test_corpus['percentage'] = (stat_test_corpus.num * 100 / stat_test_corpus.num.sum()).round(1).astype(str) + '%'
stat_test_corpus

,num,percentage
label,,
L,195,53.3%
I,171,46.7%


In [18]:
masked = []
for index, row in corpus_test.iterrows():
    masked.append(row['sentence'].replace(row['mwe'], '[MASK]'))

In [19]:
print(corpus_test['mwe'].iloc[1])
print(corpus_test['sentence'].iloc[1])

против течения
 Димитрий понял, что против течения не поплывешь, и, молча, сидел в углу.


In [20]:
corpus_test['sentence'] = masked
corpus_test['sentence'].iloc[1]

<ipython-input-20-ec4885d8cca2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus_test['sentence'] = masked


' Димитрий понял, что [MASK] не поплывешь, и, молча, сидел в углу.'

In [21]:
corpus_test.head()

,mwe,label,sentence,type
40,против течения,I,"Но -- надо отдать справедливость -- в ""Петерб...",PP
41,против течения,I,"Димитрий понял, что [MASK] не поплывешь, и, м...",PP
42,против течения,I,"ервое место поставить личность, то непременно ...",PP
43,против течения,I,оей спиной и почти одновременно надели береты....,PP
44,против течения,I,"Но -- надо отдать справедливость -- в ""Петерб...",PP


#RuBERT

In [ ]:
!pip install datasets

In [23]:
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

In [24]:
import seaborn as sns
from tqdm.notebook import tqdm
import numpy as np
import warnings
import random
import pickle

import torch
from torch.utils.data import DataLoader

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, classification_report
from sklearn import preprocessing
from transformers import AutoTokenizer, AutoModel

In [25]:
train = Dataset.from_pandas(corpus_train)
test = Dataset.from_pandas(corpus_test)

In [26]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [27]:
train_labels = le.transform(train['label'])
test_labels = le.transform(test['label'])

In [28]:
train

Dataset({
    features: ['mwe', 'label', 'sentence', 'type', '__index_level_0__'],
    num_rows: 1269
})

In [29]:
test

Dataset({
    features: ['mwe', 'label', 'sentence', 'type', '__index_level_0__'],
    num_rows: 366
})

In [30]:
rubert_tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
rubert = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
rubert_tokenizer.mask_token, rubert_tokenizer.mask_token_id

('[MASK]', 103)

In [32]:
corpus_test['sentence'].iloc[200]

'В р. Снастной тоже много лососей. Они стоят в воде [MASK], едва помахивая плавниками, а потом вдруг делают резкий бросок - выпрыгивают из воды, показывая свой яркий красный горб, и взбивают хвостом воду, а потом снова тишина.\nКрест и металлическая плита находятся на склоне сопки. Именно в этом районе командор ступил на землю и провел свои последние дни в ямах, вырытых для укрытия от ветра и снега. Поскольку могила Беринга не найдена, крест и плита воздвигнуты в условном месте. Неподалеку находятся плиты с именами погибших здесь моряков - участников экспедиции.\nВ этой прибрежной долине '

In [33]:
print(rubert_tokenizer.encode(corpus_test['sentence'].iloc[200]))
print(rubert_tokenizer.decode(rubert_tokenizer.encode(corpus_test['sentence'].iloc[200])))
print(corpus_test['mwe'].iloc[4])

[101, 781, 873, 132, 28177, 2194, 1700, 15291, 6379, 97547, 1538, 132, 9621, 27476, 845, 19962, 103, 128, 24739, 3910, 29106, 22779, 95076, 128, 625, 11302, 38987, 26253, 52021, 60075, 130, 65418, 11288, 1703, 12240, 128, 63199, 7462, 51999, 25122, 112272, 128, 851, 70494, 11288, 61426, 18253, 128, 625, 11302, 12465, 80064, 132, 18312, 851, 85910, 61292, 11841, 1469, 37013, 117937, 132, 16297, 845, 3491, 6035, 106087, 18056, 1506, 1469, 19590, 851, 16096, 8305, 12072, 16590, 845, 99182, 884, 128, 119284, 7656, 884, 2748, 73567, 1641, 28036, 851, 39740, 132, 18509, 40909, 65195, 626, 1699, 39802, 128, 12294, 851, 61292, 67094, 880, 845, 27453, 866, 9303, 132, 104314, 11841, 46920, 869, 36431, 15957, 10378, 32858, 130, 13121, 18051, 132, 781, 6377, 59527, 24187, 102]
[CLS] В р. Снастной тоже много лососей. Они стоят в воде [MASK], едва помахивая плавниками, а потом вдруг делают резкий бросок - выпрыгивают из воды, показывая свой яркий красный горб, и взбивают хвостом воду, а потом снова 

In [34]:
torch.cuda.is_available()

True

In [35]:
device_cuda = torch.device("cuda")
device_cpu = torch.device("cpu")

In [36]:
def vectorize_dataset(data, tokenizer, model):
    model = model.to(device_cuda)
    res_cls = []
    res_mean = []
    for batch in tqdm(data):
        toks = tokenizer(batch['sentence'], padding='max_length', truncation=True, return_tensors='pt', max_length=300)
        with torch.no_grad():
            model_output = model(**{k: v.to(model.device) for k, v in toks.items()})
        res_cls.append(model_output.last_hidden_state[:,0,:]) #достаём вектора для CLS токенов
        res_mean.append(torch.mean(model_output.last_hidden_state, dim=1)) #среднее
    res_cls = torch.vstack(res_cls)
    res_mean = torch.vstack(res_mean)
    return res_cls.detach().cpu().numpy(), res_mean.detach().cpu().numpy()

In [37]:
dl_train = DataLoader(train, batch_size=256, shuffle=False,
                drop_last=False)
dl_test = DataLoader(test, batch_size=256, shuffle=False,
                drop_last=False)
vecs_train_cls, vecs_train_mean = vectorize_dataset(dl_train, rubert_tokenizer, rubert)
vecs_test_cls, vecs_test_mean = vectorize_dataset(dl_test, rubert_tokenizer, rubert)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [38]:
vectors = {'vecs_train_cls' : vecs_train_cls, 'vecs_train_mean' : vecs_train_mean,
           'vecs_test_cls' : vecs_test_cls, 'vecs_test_mean' : vecs_test_mean}

for name in vectors.keys():
    with open(f'{name}.pkl', 'wb') as file:
         pickle.dump(vectors[name], file)

In [39]:
vector_names = ['vecs_train_cls', 'vecs_train_mean', 'vecs_test_cls', 'vecs_test_mean']
vecs = []

for name in vector_names:
    with open(f'{name}.pkl', "rb") as fIn:
        vecs.append(pickle.load(fIn))

train_cls = vecs[0]
train_mean = vecs[1]
test_cls = vecs[2]
test_mean = vecs[3]

In [40]:
train_cls.shape

(1269, 768)

In [41]:
test_mean.shape

(366, 768)

In [42]:
def calculate_metrics(true_labels, predictions):
    matrix = confusion_matrix(true_labels, predictions)
    acc = accuracy_score(true_labels, predictions)
    prec = precision_score(true_labels, predictions)
    rec = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    roc_auc = roc_auc_score(true_labels, predictions)
    return matrix, acc, prec, rec, f1, roc_auc

In [43]:
def baseline_rand_vec(vecs):
    size = vecs.shape[1]
    num_vecs = vecs.shape[0]
    rand_vecs = []
    for _ in range(num_vecs):
        rand_vecs.append(np.random.uniform(-5.0826163,1.5603778,size))
    return rand_vecs

def baseline_rand_pred(vecs, labels):
    num_vecs = vecs.shape[0]
    rand_vecs = []
    rand_pred = np.array(range(num_vecs), dtype=int)
    for i in range(0, num_vecs):
        rand_pred[i] = random.choice(labels)
    return rand_pred

In [44]:
def rand_pred_func(vecs, labels, true_labels, n=50):
    results_rand_pred = []
    for _ in range(n):
        pred_labels = baseline_rand_pred(vecs, labels)
        results_rand_pred.append(calculate_metrics(true_labels, pred_labels))
    res_rand_pred = pd.DataFrame(results_rand_pred, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
    res_rand_pred.to_csv('res_rand_pred.csv')

In [45]:
def get_results(train_data, train_labels, test_data, test_labels):
    results_train = []
    results_test = []
    for n in tqdm(range(50)):
        clf = MLPClassifier(random_state=n, max_iter=300, verbose=1)
        clf.fit(train_data, train_labels)
        train_pred = clf.predict(train_data)
        test_pred = clf.predict(test_data)
        results_train.append(calculate_metrics(train_labels, train_pred))
        results_test.append(calculate_metrics(test_labels, test_pred))
    return results_train, results_test, test_pred

In [46]:
labels = [0, 1]

In [47]:
rand_pred_func(test_cls, labels, test_labels)

In [ ]:
rand_vecs = baseline_rand_vec(train_cls)
results_train_rand, results_test_rand, test_pred_rand = get_results(rand_vecs, train_labels, test_cls, test_labels)
res_test_rand = pd.DataFrame(results_test_rand, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_train_rand = pd.DataFrame(results_train_rand, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_test_rand.to_csv('res_test_rand_vec.csv')
res_train_rand.to_csv('res_train_rand_vec.csv')

In [ ]:
results_train_mean, results_test_mean, test_pred_mean = get_results(train_mean, train_labels, test_mean, test_labels)
res_test_mean = pd.DataFrame(results_test_mean, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_train_mean = pd.DataFrame(results_train_mean, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_test_mean.to_csv('res_test_mean.csv')
res_train_mean.to_csv('res_train_mean.csv')

In [50]:
print(classification_report(test_labels, test_pred_mean, target_names=le.classes_))

              precision    recall  f1-score   support

           I       0.63      0.82      0.72       171
           L       0.79      0.58      0.67       195

    accuracy                           0.69       366
   macro avg       0.71      0.70      0.69       366
weighted avg       0.72      0.69      0.69       366



In [ ]:
results_train_cls, results_test_cls, test_pred_cls = get_results(train_cls, train_labels, test_cls, test_labels)
res_test_cls = pd.DataFrame(results_test_cls, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_train_cls = pd.DataFrame(results_train_cls, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_test_cls.to_csv('res_test_cls.csv')
res_train_cls.to_csv('res_train_cls.csv')

In [52]:
print(classification_report(test_labels, test_pred_cls, target_names=le.classes_))

              precision    recall  f1-score   support

           I       0.66      0.75      0.70       171
           L       0.75      0.66      0.70       195

    accuracy                           0.70       366
   macro avg       0.70      0.71      0.70       366
weighted avg       0.71      0.70      0.70       366



In [53]:
results = corpus_test

results['pred_mean'] = test_pred_mean
results['pred_cls'] = test_pred_cls
results['new_label'] = test_labels

<ipython-input-53-063455641a73>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['pred_mean'] = test_pred_mean
<ipython-input-53-063455641a73>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['pred_cls'] = test_pred_cls
<ipython-input-53-063455641a73>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [54]:
results.to_csv('results.csv')

In [55]:
results_PP = results[results['type'] == 'PP']
results_VP = results[results['type'] == 'VP']
results_NP = results[results['type'] == 'NP']

In [56]:
metrics = [calculate_metrics(results_PP['new_label'], results_PP['pred_mean']),
           calculate_metrics(results_PP['new_label'], results_PP['pred_cls']),
           calculate_metrics(results_VP['new_label'], results_VP['pred_mean']),
           calculate_metrics(results_VP['new_label'], results_VP['pred_cls']),
           calculate_metrics(results_NP['new_label'], results_NP['pred_mean']),
           calculate_metrics(results_NP['new_label'], results_NP['pred_cls'])]
groups = ['PP_mean', 'PP_cls', 'VP_mean', 'VP_cls', 'NP_mean', 'NP_cls']

In [57]:
all_metrics = pd.DataFrame(metrics, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
all_metrics['group'] = groups
all_metrics = all_metrics[['group', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc']]
all_metrics

,group,accuracy,precision,recall,f1,roc_auc
0,PP_mean,0.680769,0.859813,0.575,0.689139,0.712500
1,PP_cls,0.715385,0.811594,0.700,0.751678,0.720000
2,VP_mean,0.728814,0.680000,0.680,0.680000,0.722353
3,VP_cls,0.694915,0.666667,0.560,0.608696,0.677059
4,NP_mean,0.750000,0.300000,0.600,0.400000,0.687097
5,NP_cls,0.666667,0.181818,0.400,0.250000,0.554839


In [58]:
all_metrics.to_csv('group_metrics.csv')

In [59]:
mwe_res = dict()

for mwe in results.groupby('mwe').__iter__():
    mwe_res.update({mwe[0] : {'mean' : accuracy_score(mwe[1]['new_label'], mwe[1]['pred_mean']),
                           'cls' : accuracy_score(mwe[1]['new_label'], mwe[1]['pred_cls'])}})

In [60]:
results['mwe'].value_counts()

mwe
против течения            141
не на своем месте          45
под каблуком               35
на свою голову             34
гладить по голове          30
удар ниже пояса            22
жирно будет                12
наш брат                   10
за пояс заткнут             9
каши не свариш              6
голова болит                6
на мою голову               5
не за горами                5
косой взгляд                4
балансировать на грани      2
Name: count, dtype: int64

In [61]:
idiom_res = pd.DataFrame.from_dict(mwe_res).transpose()
idiom_res

,mean,cls
балансировать на грани,1.000000,1.000000
гладить по голове,0.633333,0.633333
голова болит,0.333333,0.333333
жирно будет,1.000000,0.833333
за пояс заткнут,0.555556,0.555556
каши не свариш,0.833333,0.833333
косой взгляд,1.000000,0.250000
на мою голову,0.600000,0.600000
на свою голову,0.588235,0.529412
наш брат,0.600000,0.400000


In [62]:
idiom_res.to_csv('idiom_results.csv')

In [67]:
res_rand_pred = pd.read_csv('/content/res_rand_pred.csv')

In [70]:
all_test = [res_rand_pred.iloc[:, 2:].mean(), res_test_rand.iloc[:, 2:].mean(),
            res_test_cls.iloc[:, 2:].mean(), res_test_mean.iloc[:, 2:].mean()]
all_train = [res_train_rand.iloc[:, 2:].mean(), res_train_cls.iloc[:, 2:].mean(), res_train_mean.iloc[:, 2:].mean()]

In [71]:
pd.concat(all_test, axis=1).rename(columns={0:'rand_pred', 1:'test_rand_vec', 2:'test_cls', 3:'test_mean'})

,rand_pred,test_rand_vec,test_cls,test_mean
accuracy,0.496831,NaN,NaN,NaN
precision,0.529817,0.500702,0.737389,0.772052
recall,0.496718,0.515590,0.673128,0.626359
f1,0.512452,0.441295,0.703644,0.691230
roc_auc,0.496839,0.521655,0.699664,0.707507


In [72]:
pd.concat(all_train, axis=1).rename(columns={0:'train_rand_vec', 1:'train_cls', 2:'train_mean'})

,train_rand_vec,train_cls,train_mean
precision,1.0,0.999451,0.999095
recall,1.0,0.998932,0.999288
f1,1.0,0.999191,0.999191
roc_auc,1.0,0.999205,0.999214


In [73]:
!zip -r /content/file.zip /content/russian_all_mask

  adding: content/russian_all_mask/ (stored 0%)
  adding: content/russian_all_mask/res_train_mean.csv (deflated 94%)
  adding: content/russian_all_mask/group_metrics.csv (deflated 50%)
  adding: content/russian_all_mask/res_train_cls.csv (deflated 94%)
  adding: content/russian_all_mask/vecs_train_cls.pkl (deflated 15%)
  adding: content/russian_all_mask/idiom_results.csv (deflated 55%)
  adding: content/russian_all_mask/res_rand_pred.csv (deflated 68%)
  adding: content/russian_all_mask/vecs_test_cls.pkl (deflated 16%)
  adding: content/russian_all_mask/results.csv (deflated 74%)
  adding: content/russian_all_mask/res_test_mean.csv (deflated 70%)
  adding: content/russian_all_mask/vecs_train_mean.pkl (deflated 15%)
  adding: content/russian_all_mask/vecs_test_mean.pkl (deflated 16%)
  adding: content/russian_all_mask/res_test_rand_vec.csv (deflated 64%)
  adding: content/russian_all_mask/res_train_rand_vec.csv (deflated 92%)
  adding: content/russian_all_mask/res_test_cls.csv (deflate